#### import 需要用的 library

In [1]:
pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.5 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=6404958a4aca299f9443568fdee077941bfb36939c99a4ff777e08fae3495221
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=9901dde9ac3431eee5d9521fd88c06a228d7689f63317eb225f0e6c9bbacaa9e
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/{你的google雲端硬碟資料夾位址}') #切換該目錄

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/IP/final') #切換該目錄

In [4]:
from segmentation_models_pytorch import Unet
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torchvision.transforms.functional as TF
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
import re

#### 初始化一些定義內容

更改參數都在這裡

In [5]:
### init parameter
# Constants for UNet model training process
BATCH_SIZE = 2
IMG_HEIGHT = 512
IMG_WIDTH = 512
NUM_EPOCHS = 100
Learning_Ratio = 1e-3
THRESOLD = 0.5

# Load data
test_img_dir = r'testing_dataset/image'
test_mask_dir = r'testing_dataset/mask'
train_img_dir = r'training_dataset/image'
train_mask_dir = r'training_dataset/mask'
# model_load_file = r'unet_25_test1_iou_0632_origin'

#### 一些有用的function 寫在這裡
 計算IOU...

In [6]:
def calculate_iou_tensor(image1, image2, threshold=THRESOLD):
    """
    計算兩個 PyTorch Tensor 的交集比（IoU）。

    :param image1: 第一張影像（PyTorch Tensor）
    :param image2: 第二張影像（PyTorch Tensor）
    :param threshold: 灰階二值化的閾值（預設 128）
    :return: IoU 值（介於 0 到 1）
    """
    # 確保兩張影像的形狀相同

    if image1.shape != image2.shape:
        raise ValueError("兩張影像必須具有相同的尺寸！")
    # 將灰階影像二值化
    binary1 = (image1 >= threshold).to(torch.uint8)
    binary2 = (image2 >= threshold).to(torch.uint8)

    # 計算交集和聯集
    intersection = torch.sum(binary1 & binary2).item()
    union = torch.sum(binary1 | binary2).item()

    # 防止分母為零
    if union == 0:
        return 0.0

    # 計算 IoU
    iou = intersection / union
    return iou

##### Data Augmentation (資料增強)
 做一些rotatation、hflip、vflip、Gaussian Blur、資料對比增強 等等

In [7]:
def extract_number(filename):
    """
    從檔名中提取第一個出現的數字並轉換為整數。
    如果找不到數字，則返回一個很大的數，使該檔案排在最後。
    """
    name, _ = os.path.splitext(filename)
    match = re.search(r'\d+', name)
    if match:
        return int(match.group())
    else:
        return float('inf')  # 沒有數字的檔案將排在最後


# 自定義變換類別
class ImageRotateFlipTransform:
    def __call__(self, img, mask):
        # 同步水平翻轉
        if random.random() > 0.5:
            img = T.functional.hflip(img)
            mask = T.functional.hflip(mask)
        if random.random() > 0.5:
            img = T.functional.vflip(img)
            mask = T.functional.vflip(mask)
        # 同步旋轉
        angles = [0,90,270]
        # angles = [0]
        angle = random.choice(angles)  # 隨機選擇旋轉角度
        img = T.functional.rotate(img, angle)
        mask = T.functional.rotate(mask, angle)

        return img, mask

class customDataSet(Dataset):
  def __init__(self, img_dir, mask_dir, transform_img,transform_mask,Rotate_Flag=True):
    super().__init__()
    self.img_dir = img_dir
    self.mask_dir = mask_dir
    self.transform_img = transform_img
    self.transform_mask = transform_mask
    self.images = [f for f in os.listdir(img_dir)] #僅讀取副檔名為以下的
    self.images = sorted(self.images, key=extract_number)
    self.masks  = [f for f in os.listdir(mask_dir)]
    self.masks  = sorted(self.masks, key=extract_number)
    self.rotate_flag =  Rotate_Flag
    self.ImageRotateFlipTransform = ImageRotateFlipTransform()
  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.images[idx])
    mask_path = os.path.join(self.mask_dir, self.masks[idx])
    image = Image.open(img_path).convert('RGB')
    mask = Image.open(mask_path).convert('L') # 轉成黑白圖片 0 或 255

    if self.rotate_flag :
        image , mask = self.ImageRotateFlipTransform(image,mask)
    if self.transform_img and self.transform_mask:
       image = self.transform_img(image)
       mask  = self.transform_mask(mask)


    return image,mask

#### setting DataLoader

loading moel 使用Unet 並且 backbone 採用 resenet34 ， 不使用 pre-train 參數  

設定train 裡面要做的 data augmentation 的一些方法，並且resize成一樣的大小

In [8]:

# model_load_file = None
# 使用 ResNet34 作為 UNet 的 backbone
model = Unet(encoder_name="resnet34",
             encoder_weights=None,  # 不用pre-train的權重
             in_channels=3,
             classes=1,
             )

# Check the device we are using is GPU or CPU
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Use the GPU to train')
else:
  device = torch.device('cpu')
  print('Use the CPU to train')
model = model.to(device)

transform_img = T.Compose([
                T.ToTensor(),                                  # 轉換為 Tensor
                # T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                T.Resize((IMG_HEIGHT, IMG_WIDTH)),            # 調整圖片大小
                T.RandomApply([
                    T.GaussianBlur(kernel_size=7)
                ], p=1),                                   # 以 50% 機率進行模糊
                T.RandomApply([
                    T.ColorJitter(
                        brightness=0.4,                           # 隨機調整亮度 (0.8 ~ 1.2 範圍)
                        contrast=0.5,                             # 隨機調整對比度
                        saturation=0.2,                           # 飽和度不變 (設為 0)
                        hue=0.1                                   # 色調不變
                    )             # 隨機高斯模糊 (核大小設為 5)
                ],p=1)
            ])

# valid 都不做resize 直接算原始的 iou 大小
transform_test_img = T.Compose([T.ToTensor(),
                  # T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                  T.Resize((IMG_HEIGHT, IMG_WIDTH)),
                  T.GaussianBlur(kernel_size=3)
                  ])
transform_mask  = T.Compose([T.ToTensor(),
                T.Resize((IMG_HEIGHT, IMG_WIDTH)),
                ])
train_data    = customDataSet(train_img_dir, train_mask_dir,transform_img,transform_mask,Rotate_Flag=True)
valid_dataset = customDataSet(test_img_dir, test_mask_dir ,transform_test_img,transform_mask,Rotate_Flag=False)

train_loader  = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,pin_memory=True)
valid_loader  = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False,pin_memory=True)

loss_function = nn.BCEWithLogitsLoss() # 內部會幫忙做sigmoid
optimizer = optim.Adam(model.parameters(),lr=Learning_Ratio) # Choosing Adam as our optimizer
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5) # new_lr = lr*factor

Use the GPU to train


#### save、load model function

In [9]:
def save_model(model, optimizer, epoch, best_iou, save_path):
    """
    保存模型的狀態字典和優化器的狀態字典。
    :param model: 要保存的模型
    :param optimizer: 優化器
    :param epoch: 當前 epoch
    :param best_iou: 當前最佳的 IoU 分數
    :param save_path: 模型保存的路徑
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_iou': best_iou
    }
    torch.save(checkpoint, save_path)
    print(f"模型已保存到 {save_path}")


### 定義train function

In [10]:
def train(model, num_epochs, train_loader,valid_loader, optimizer,loss_function,device):
    model.to(device)
    best_iou = 0
    best_acc = 0
    for epoch in range(num_epochs):
        num_correct = 0
        num_pixels = 0
        total_loss = 0
        ################## train model
        model.train()
        for count, (x, y) in enumerate(train_loader):
            x = x.to(device) # [BATCH_SIZE , CHANNEL , WIDTH , HEIGHT]
            y = y.to(device) # [BATCH_SIZE , CHANNEL , WIDTH , HEIGHT]
            out = model(x) # out range [-inf , inf]
            ############ ACC #######
            # out = torch.sigmoid(out) # loss 要是BCELoss
            loss = loss_function(out, y)
            total_loss += loss.item()
            predictions = (torch.sigmoid(out) >= THRESOLD) # 預測出來的mask樣子
            origin_mask = (y>=THRESOLD)     # 原本train_set 裡面的 mask
            num_correct += (predictions==origin_mask).sum().item()
            # num_pixels += BATCH_SIZE*IMG_WIDTH*IMG_HEIGHT
            num_pixels += BATCH_SIZE*origin_mask.size(2)*origin_mask.size(3)
            ##############################
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        acc_mean = num_correct/num_pixels
        avg_loss = total_loss / len(train_loader)
        if best_acc <= acc_mean:
           best_acc = acc_mean
           save_model(model, optimizer, epoch, best_iou, f"unet_{epoch+1}_test1_acc")
        print(f'Epoch[{epoch+1}] Acc: {acc_mean:.3f} - AvgLoss: {avg_loss:.3f}')
        ################# eval model
        model.eval()
        with torch.no_grad():
            iou_total = 0
            total_number_picture = 0
            valid_loss = 0
            for count, (x, y) in enumerate(valid_loader):
                batch_size = x.size(0)
                total_number_picture += batch_size
                x = x.to(device)
                y = y.to(device)
                out  = model(x)
                ###### 計算驗證損失
                loss = loss_function(out, y)
                valid_loss += loss.item()
                ###############################
                out = torch.sigmoid(out) # loss 要是BCELoss
                for i in range(batch_size):
                    # 分別取出第 i 張影像及對應的預測與標註遮罩
                    # 注意確保維度匹配你的 iou_function 輸入格式
                    img_single = x[i].squeeze(0)    # [C, H, W] -> [1, C, H, W]
                    mask_single = y[i].squeeze(0)   # [H, W] -> [1, H, W] 或 [1, 1, H, W]

                    predic_mask = out[i].squeeze(0) # [C, H, W] -> [1, C, H, W]

                    # 利用你的 iou 函式計算該張影像的 IoU
                    iou_total += calculate_iou_tensor(mask_single,predic_mask)

        avg_valid_loss = valid_loss / len(valid_loader)
        scheduler.step(avg_valid_loss)  # 使用驗證損失來調整學習率

        iou_mean = (iou_total/total_number_picture)
        if best_iou <= iou_mean:
            best_iou = iou_mean
            save_model(model, optimizer, epoch, best_iou, f"unet_{epoch+1}_test1_iou")
        print(f'Epoch[{epoch+1}] - Test | IOU_Mean : {iou_mean:.3f} | AvgLoss : {avg_valid_loss:.3f} ')


In [11]:
# start to train
train(model, NUM_EPOCHS, train_loader,valid_loader,optimizer,loss_function,device)

模型已保存到 unet_1_test1_acc
Epoch[1] Acc: 0.606 - AvgLoss: 0.642
模型已保存到 unet_1_test1_iou
Epoch[1] - Test | IOU_Mean : 0.326 | AvgLoss : 0.673 
模型已保存到 unet_2_test1_acc
Epoch[2] Acc: 0.634 - AvgLoss: 0.633
模型已保存到 unet_2_test1_iou
Epoch[2] - Test | IOU_Mean : 0.385 | AvgLoss : 0.789 
Epoch[3] Acc: 0.572 - AvgLoss: 0.666
Epoch[3] - Test | IOU_Mean : 0.366 | AvgLoss : 0.645 


KeyboardInterrupt: 